#import guardrails

In [1]:
import os
from hr__guardrails import (
    violates_safety_policy,
    is_hr_question,
    is_answer_grounded,
    contains_sensitive_advice,
    is_query_too_long
)


Load PDFs (with OCR fallback)

In [2]:
from langchain_community.document_loaders import PyPDFLoader, UnstructuredPDFLoader

def load_all_pdfs(pdf_folder_path):
    documents = []

    for file_name in os.listdir(pdf_folder_path):
        if not file_name.endswith(".pdf"):
            continue

        full_path = os.path.join(pdf_folder_path, file_name)

        loader = PyPDFLoader(full_path)
        pages = loader.load()

        extracted_text = "".join([p.page_content.strip() for p in pages])

        if not extracted_text:
            print(f"OCR applied for: {file_name}")
            loader = UnstructuredPDFLoader(full_path, strategy="ocr_only")
            pages = loader.load()

        for doc in pages:
            doc.metadata["source"] = file_name

        documents.extend(pages)

    return documents


pdf_folder_path = r"C:\Users\91880\Desktop\GenAI_Projects\PuchoHR\Data\documents"
docs = load_all_pdfs(pdf_folder_path)
print("Total pages:", len(docs))


C:\Users\91880\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Total pages: 72


Chunking

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=256
)

chunks = text_splitter.split_documents(docs)
print("Total chunks:", len(chunks))


Total chunks: 526


Create & Persist Vector DB

In [4]:
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

embedding = OllamaEmbeddings(model="nomic-embed-text")

vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=embedding,
    persist_directory="./chroma_db"
)

vector_db.persist()
print("Vector DB created and persisted.")


Vector DB created and persisted.


C:\Users\91880\AppData\Local\Temp\ipykernel_16376\3096279755.py:12: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_db.persist()


Load Vector DB + Models

In [5]:
from sentence_transformers import SentenceTransformer, util
from langchain_ollama import OllamaLLM

embedding_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

vector_db = Chroma(
    persist_directory="./chroma_db",
    embedding_function=embedding
)

llm = OllamaLLM(
    model="llama3.2",
    temperature=0.2
)


C:\Users\91880\AppData\Local\Temp\ipykernel_16376\1718535300.py:6: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vector_db = Chroma(


HR Bot Function

In [6]:
def ask_hr(question):
    if is_query_too_long(question):
        return "Your query is too long, please shorten it."

    if violates_safety_policy(question):
        return "Sorry, I cannot answer that question due to safety policy."

    if not is_hr_question(question):
        return "Sorry, this question is not related to HR policies."

    docs = vector_db.similarity_search(question, k=4)
    if not docs:
        return "Information not found in documents."

    context = "\n\n".join(
        f"Source: {doc.metadata['source']}\n{doc.page_content}"
        for doc in docs
    )

    prompt = f"""
You are an HR assistant.
Answer ONLY using the provided context.
If the answer is not present, say:
"Information not found in documents."

Context:
{context}

Question:
{question}

Answer:
"""

    answer = llm.invoke(prompt)

    if not is_answer_grounded(answer, docs):
        return "Information not found in documents."

    answer_emb = embedding_model.encode(str(answer))
    context_text = " ".join(doc.page_content for doc in docs)
    context_emb = embedding_model.encode(context_text)
    similarity = util.cos_sim(answer_emb, context_emb).item()

    if similarity < 0.8:
        return "Answer may not be fully accurate based on available documents."

    if contains_sensitive_advice(answer):
        return "I cannot provide legal or medical advice. Please consult the appropriate professional."

    return answer


Database + Test

In [7]:
from db_util import init_db, store_chat, get_recent_chat

init_db()

session_id = "user123"
question = "What is the leave policy for regular employees?"

answer = ask_hr(question)
print("LLM answer:", answer)

store_chat(session_id, question, answer, source_doc="HR_Policy-GGIAL.pdf", similarity_score=0.92)

previous_chats = get_recent_chat(session_id)
print(previous_chats)


LLM answer: Answer may not be fully accurate based on available documents.
[('What is the leave policy for regular employees?', 'Answer may not be fully accurate based on available documents.'), ('What is the leave policy for regular employees?', 'According to the HR Policy-GGIAL.pdf, the leave policies for regular employees are as follows:\n\n* Sick Leave: 6 days per year; Medical Proof of leave exceeding 2 days must be submitted by the employee. No limit on accumulation, but it cannot be encashed. SL can be availed for half day also.\n* Privilege Leave: 24 days per year; Maximum Accumulation is up to 180 days. PL can be encased only at the time of separation, and monthly basic salary will be considered for calculation.')]


In [8]:
# Ask multiple questions
q1 = "What is the leave policy for regular employees?"
a1 = ask_hr(q1)
store_chat("user123", q1, a1, source_doc="HR_Policy-GGIAL.pdf", similarity_score=0.90)

q2 = "What is the probation period?"
a2 = ask_hr(q2)
store_chat("user123", q2, a2, source_doc="HR_Policy-GGIAL.pdf", similarity_score=0.88)

print("Q1:", a1)
print("Q2:", a2)

# Fetch last chats
history = get_recent_chat("user123", limit=5)
print("\nRecent Chat History:")
for q, a in history:
    print("Q:", q)
    print("A:", a)
    print("-" * 50)


Q1: Answer may not be fully accurate based on available documents.
Q2: Information not found in documents.

Recent Chat History:
Q: What is the probation period?
A: Information not found in documents.
--------------------------------------------------
Q: What is the leave policy for regular employees?
A: Answer may not be fully accurate based on available documents.
--------------------------------------------------
Q: What is the leave policy for regular employees?
A: Answer may not be fully accurate based on available documents.
--------------------------------------------------
Q: What is the leave policy for regular employees?
A: According to the HR Policy-GGIAL.pdf, the leave policies for regular employees are as follows:

* Sick Leave: 6 days per year; Medical Proof of leave exceeding 2 days must be submitted by the employee. No limit on accumulation, but it cannot be encashed. SL can be availed for half day also.
* Privilege Leave: 24 days per year; Maximum Accumulation is up to